In [8]:
import math
import csv
from collections import defaultdict

class Label:
    def __init__(self, w_x, w_y, path, visited):
        self.w_x = w_x          # Sum of positive weights (x dimension)
        self.w_y = w_y          # Sum of absolute negative weights (y dimension)
        self.path = path        # Path as list of nodes
        self.visited = visited  # Frozen set of visited nodes

    def __repr__(self):
        return f"Label(w_x={self.w_x:.2f}, w_y={self.w_y:.2f}, path={self.path})"

def load_graph_from_csv(filename):
    """Loads graph from CSV with format: u,v,weight"""
    graph = defaultdict(list)
    max_weight = 0
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header
        for row in reader:
            u, v, weight = row
            weight = float(weight)
            
            # Convert to 2D weight representation
            if weight >= 0:
                wx, wy = weight, 0.0
                max_weight = max(max_weight, weight)
            else:
                wx, wy = 0.0, abs(weight)
            
            graph[u].append((v, wx, wy))
    return graph, max_weight

def additive_fptas_shortest_path(csv_file, source, target, epsilon):
    # Load graph and get maximum positive weight
    graph, W = load_graph_from_csv(csv_file)
    nodes = list(graph.keys())
    n = len(nodes)
    if n == 0:
        return float('inf'), []
    
    # Calculate discretization parameter
    delta = epsilon / (n-1) if n > 1 else epsilon
    max_interval = int((n-1)*W/delta) + 1 if W > 0 else 1

    # Initialize data structures
    labels = defaultdict(dict)  # {node: {interval: Label}}
    initial_visited = frozenset([source])
    labels[source][0] = Label(0.0, 0.0, [source], initial_visited)

    # Main label propagation loop
    for _ in range(n-1):
        new_labels = defaultdict(dict)
        
        for u in labels:
            for interval in labels[u]:
                current_label = labels[u][interval]
                
                for (v, wx_e, wy_e) in graph.get(u, []):
                    # Skip already visited nodes
                    if v in current_label.visited:
                        continue
                    
                    # Calculate new weights
                    new_wx = current_label.w_x + wx_e
                    new_wy = current_label.w_y + wy_e
                    
                    # Calculate interval using uniform discretization
                    interval_new = int(new_wx // delta)
                    
                    # Update visited nodes
                    new_visited = current_label.visited.union(frozenset([v]))
                    
                    # Create new path
                    new_path = current_label.path + [v]
                    
                    # Create new label
                    new_label = Label(new_wx, new_wy, new_path, new_visited)
                    
                    # Update if better than existing label in this interval
                    existing = new_labels[v].get(interval_new)
                    if not existing or new_label.w_y > existing.w_y:
                        new_labels[v][interval_new] = new_label

        # Merge new labels into main structure
        for node in new_labels:
            for interval in new_labels[node]:
                if interval not in labels[node] or \
                   new_labels[node][interval].w_y > labels[node][interval].w_y:
                    labels[node][interval] = new_labels[node][interval]

    # Find best path to target
    min_value = float('inf')
    best_path = []
    for interval in labels.get(target, {}):
        label = labels[target][interval]
        current_value = label.w_x - label.w_y
        if current_value < min_value:
            min_value = current_value
            best_path = label.path

    return min_value, best_path

# Example usage
if __name__ == "__main__":
    CSV_FILE = "graph_data.csv"
    SOURCE = "n0"
    TARGET = "n49"  # Update with your target node
    EPSILON = 0.1   # Example epsilon value
    
    min_value, path = additive_fptas_shortest_path(CSV_FILE, SOURCE, TARGET, EPSILON)
    
    if path:
        print(f"Optimal value: {min_value:.2f}")
        print(f"Path: {' -> '.join(path)}")
    else:
        print("No valid path exists")

Optimal value: -282.00
Path: n0 -> n2 -> n12 -> n47 -> n29 -> n42 -> n24 -> n44 -> n8 -> n9 -> n22 -> n15 -> n21 -> n38 -> n19 -> n16 -> n18 -> n36 -> n13 -> n23 -> n6 -> n41 -> n5 -> n4 -> n32 -> n10 -> n37 -> n34 -> n1 -> n7 -> n33 -> n46 -> n28 -> n20 -> n35 -> n45 -> n40 -> n14 -> n27 -> n39 -> n49


In [7]:
import csv
import math
from collections import defaultdict, deque

def load_graph_from_csv(filename):
    graph = defaultdict(dict)
    
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)  # Skip header
        
        for row in reader:
            source, target, weight = row
            weight = int(weight)
            graph[source][target] = weight
    
    return graph

class AdditiveFPTAS:
    def __init__(self, graph, source, target, epsilon):
        self.graph = graph
        self.source = source
        self.target = target
        self.epsilon = epsilon
        self.n = len(graph)
        self.delta = epsilon / (self.n - 1)
        
        # Transform the graph to bi-objective format
        self.bi_graph = self.transform_graph()
    
    def transform_graph(self):
        bi_graph = defaultdict(dict)
        for u in self.graph:
            for v, weight in self.graph[u].items():
                if weight >= 0:
                    bi_graph[u][v] = (weight, 0)  # Positive weight
                else:
                    bi_graph[u][v] = (0, abs(weight))  # Negative weight as penalty
        return bi_graph
    
    def get_bucket(self, x_weight):
        if x_weight <= 0:
            return 0
        return math.floor(x_weight / self.delta)
    
    def run(self):
        # Dictionary to store paths for each node and bucket
        pareto_sets = {node: {} for node in self.graph}
        
        # Initialize the source node
        pareto_sets[self.source][0] = (0, 0, [self.source])
        
        # Queue for nodes to process
        queue = deque([self.source])
        in_queue = {self.source}
        
        while queue:
            node = queue.popleft()
            in_queue.remove(node)
            
            # Process each label at the current node
            for bucket, (x_weight, y_weight, path) in list(pareto_sets[node].items()):
                # Process each neighbor
                for neighbor, (edge_x, edge_y) in self.bi_graph[node].items():
                    # Skip if the neighbor is already in the path (avoid cycles)
                    if neighbor in path:
                        continue
                    
                    # Calculate new x and y weights
                    new_x = x_weight + edge_x
                    new_y = y_weight + edge_y
                    
                    # Get the bucket for the new x weight
                    new_bucket = self.get_bucket(new_x)
                    
                    # Check if we already have a path for this bucket
                    is_dominated = False
                    
                    if new_bucket in pareto_sets[neighbor]:
                        _, old_y, _ = pareto_sets[neighbor][new_bucket]
                        if old_y >= new_y:
                            is_dominated = True
                    
                    if not is_dominated:
                        # Update with the new path
                        new_path = path + [neighbor]
                        pareto_sets[neighbor][new_bucket] = (new_x, new_y, new_path)
                        
                        # Add the neighbor to the queue for processing
                        if neighbor not in in_queue:
                            queue.append(neighbor)
                            in_queue.add(neighbor)
        
        # Find the best path to the target
        best_value = float('inf')
        best_path = None
        
        for bucket, (x_weight, y_weight, path) in pareto_sets[self.target].items():
            value = x_weight - y_weight
            if value < best_value:
                best_value = value
                best_path = path
        
        if best_path:
            return best_path, best_value
        else:
            return None, None

def main():
    # Load the graph
    graph = load_graph_from_csv("graph_data.csv")
    
    # Set parameters
    epsilon = 0.1  # Approximation parameter
    
    # Run the FPTAS algorithm
    fptas = AdditiveFPTAS(graph, 'n0', 'n49', epsilon)
    result = fptas.run()
    
    if result[0]:
        path, value = result
        print(f"Best path: {' -> '.join(path)}")
        print(f"Path value: {value}")
    else:
        print("No path found.")

if __name__ == "__main__":
    main()


Best path: n0 -> n2 -> n12 -> n43 -> n24 -> n17 -> n4 -> n32 -> n20 -> n19 -> n16 -> n18 -> n36 -> n13 -> n23 -> n6 -> n22 -> n15 -> n21 -> n38 -> n25 -> n45 -> n40 -> n14 -> n27 -> n8 -> n9 -> n44 -> n34 -> n47 -> n29 -> n42 -> n10 -> n37 -> n33 -> n3 -> n41 -> n49
Path value: -249
